# Project 3: Poisonous Mushrooms

In this project, you'll investigate properties of mushrooms. This classic dataset contains over 8000 observations, where each mushroom is described by a variety of features like color, odor, etc., and the target variable is an indicator for whether the mushroom is poisonous. Since all the observations are categorical, I've binarized the feature space. Look at the feature_names below to see all 126 binary names.

You'll start by running PCA to reduce the dimensionality from 126 down to 2 so that you can easily visualize the data. In general, PCA is very useful for visualization (though sklearn.manifold.tsne is known to produce better visualizations). Recall that PCA is a linear transformation. The 1st projected dimension is the linear combination of all 126 original features that captures as much of the variance in the data as possible. The 2nd projected dimension is the linear combination of all 126 original features that captures as much of the remaining variance as possible. The idea of dense low dimensional representations is crucial to machine learning!

Once you've projected the data to 2 dimensions, you'll experiment with clustering using KMeans and density estimation with Gaussian Mixture Models. Finally, you'll train a classifier by fitting a GMM for the positive class and a GMM for the negative class, and perform inference by comparing the probabilities output by each model.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [0]:
%matplotlib inline

import urllib.request as urllib2 # For python3
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

# additional imports
from math import sqrt, ceil

In [0]:
MUSHROOM_DATA = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.data'
MUSHROOM_MAP = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.map'

Load feature names.

In [0]:
feature_names = []

for line in urllib2.urlopen(MUSHROOM_MAP):
    [index, name, junk] = line.decode('utf-8').split()
    feature_names.append(name)

print('Loaded feature names: ', len(feature_names))

Load data. The data is sparse in the input file, but there aren't too many features, so we'll use a dense representation, which is supported by all sklearn objects.

In [0]:
X, Y = [], []

for line in urllib2.urlopen(MUSHROOM_DATA):
    items = line.decode('utf-8').split()
    Y.append(int(items.pop(0)))
    x = np.zeros(len(feature_names))
    for item in items:
        feature = int(str(item).split(':')[0])
        x[feature] = 1
    X.append(x)

# Convert these lists to numpy arrays.
X = np.array(X)
Y = np.array(Y)

# Split into train and test data.
train_data, train_labels = X[:7000], Y[:7000]
test_data, test_labels = X[7000:], Y[7000:]

# Check that the shapes look right.
print(train_data.shape, test_data.shape)

### Part 1:

Run Principal Components Analysis on the data. Show what fraction of the total variance in the training data is explained by the first k principal components, for k in [1, 50].

In [0]:
def P1():
  pca = PCA(n_components=50)
  pca.fit(train_data)
  total_explained_variance = 0.0
  # loop through array of %variance explained for each component
  for k, component in enumerate(pca.explained_variance_ratio_):
    # add the variance of the next component
    total_explained_variance += component
    # print the sum of the first k components
    print("The first " + str(k + 1) + " Pincipal Component(s) Explain " 
          + "{:.2%}".format(total_explained_variance) + " of the variance.")

P1()

### Part 2:

PCA can be very useful for visualizing data. Project the training data down to 2 dimensions and plot it. Show the positive (poisonous) cases in blue and the negative (non-poisonous) in red. Here's a reference for plotting: http://matplotlib.org/users/pyplot_tutorial.html

In [0]:
def P2():
  # fit pca and apply dimensionality reduction on X
  pca = PCA(n_components=2)
  reduced_train_data = pca.fit_transform(train_data)
  
  # initialize plt axis object
  fig_size = 10
  fig = plt.figure(figsize=(fig_size, fig_size))
  ax = fig.add_subplot(1, 1, 1)
  
  # convert train labels to bool for use in array indexing
  label_bools = np.array(train_labels, dtype=bool)

  # plot separate series using label_bools to allow for legend labels
  ax.plot(reduced_train_data[label_bools, 0], 
          reduced_train_data[label_bools, 1], 'b+', markersize=5)
  ax.plot(reduced_train_data[~label_bools, 0], 
          reduced_train_data[~label_bools, 1], 'r+', markersize=5)
  
  # set legend, labels and title
  ax.legend(["poisonous", "non-poisonous"])
  ax.set_xlabel("PC1")
  ax.set_ylabel("PC2")
  ax.set_title("PC1 vs PC2")

  # show plot
  plt.show()


P2()

### Part 3:

Run KMeans with [1,16] clusters over the 2d projected data. Mark each centroid cluster and plot a circle that goes through the most distant point assigned to each cluster.

In [0]:
def P3():
  max_clusters = 16

  # fit pca and apply dimensionality reduction on X
  pca = PCA(n_components=2)
  reduced_train_data = pca.fit_transform(train_data)

  # initialize plt figure object with correct size given max_clusters
  plot_size = 5
  num_plots_horiz = 4
  num_plots_vert = ceil(max_clusters / 4.0)
  fig = plt.figure(figsize=(plot_size * num_plots_horiz, 
                            plot_size * num_plots_vert))

  # loop through k values and add subplots for each
  for k in range(1, max_clusters + 1):
    ax = fig.add_subplot(num_plots_vert, num_plots_horiz, k)

    # fit kmeans on reduced X
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(reduced_train_data)
    
    # get centers and max distances to centers
    centers = kmeans.cluster_centers_
    max_center_distances = np.zeros(len(centers))
    for i, label in enumerate(kmeans.labels_):
      dist = sqrt((reduced_train_data[i, 0] - centers[label, 0])**2 + 
                  ((reduced_train_data[i, 1] - centers[label, 1])**2))
      if(dist > max_center_distances[label]):
        max_center_distances[label] = dist
    
    # convert train labels to bool for use in array indexing
    label_bools = np.array(train_labels, dtype=bool)

    # plot separate series using label_bools to allow for legend labels
    ax.plot(reduced_train_data[label_bools, 0], 
            reduced_train_data[label_bools, 1], 
            'b+', label='poisonous', markersize=5)
    ax.plot(reduced_train_data[~label_bools, 0], 
            reduced_train_data[~label_bools, 1], 
            'r+', label='non-poisonous', markersize=5)
    
    # plot cluster centers and circles
    ax.plot(centers[:, 0], centers[:, 1], 'gX', markersize=10, label='clusters')
    for center, dist in zip(centers, max_center_distances):
      circle = plt.Circle(center, dist, color='g', fill=False)
      ax.add_artist(circle)
    
    # set labels and title
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_title('k = ' + str(k))

  # set legend for whole figure at once
  handles, labels = fig.axes[0].get_legend_handles_labels()
  fig.legend(handles, labels, loc='upper left', prop={'size': 20})

  # set figure title
  fig.suptitle('PC1 vs PC2 with KMeans clusters for k in [1, ' 
               + str(max_clusters) + ']', fontsize=16)

  # show plot
  plt.show()

P3()

### Part 4:

Fit a Gaussian Mixture Model for the positive examples in your 2d projected data. Plot the estimated density contours as shown here: http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html#example-mixture-plot-gmm-pdf-py. Vary the number of mixture components from 1-4 and the covariance matrix type ('spherical', 'diag', 'tied', 'full').

In [0]:
def P4():
  #initialize GMM args arrays to loop through
  components_array = range(1,5)
  covariancetype_array = ['spherical', 'diag', 'tied', 'full']

  # fit pca and apply dimensionality reduction on X
  pca = PCA(n_components=2)
  reduced_train_data = pca.fit_transform(train_data)

  # convert train labels to bool for use in array indexing
  label_bools = np.array(train_labels, dtype=bool)

  pos_train_data = reduced_train_data[label_bools]

  for cov_type in covariancetype_array:
    # initialize plt figure object with correct size given max_clusters
    plot_size = 5
    num_plots_horiz = len(components_array)
    num_plots_vert = 1
    fig = plt.figure(figsize=(plot_size * num_plots_horiz, 
                              plot_size * num_plots_vert))
    fig.suptitle('GMM with "' + cov_type + '" covariance matrix  for positive' +
                 ' samples with n_components in [1, ' 
                 + str(len(components_array)) + ']', fontsize=16)
    for n_comp in components_array:
      # add subplot
      ax = fig.add_subplot(num_plots_vert, num_plots_horiz, n_comp)
      ax.set_title('n = ' + str(n_comp))

      # fit model
      gmm = GaussianMixture(n_components=n_comp, covariance_type=cov_type)
      gmm.fit(pos_train_data)

      # get scores for mesh grid covering the domain
      # set domain limits
      x = np.linspace(-3., 3.)
      y = np.linspace(-3., 3.)
      # produce matrix of points
      Xm, Ym = np.meshgrid(x, y)
      # make array of X-vector, Y-Vector, take transpose to produce (x,y) pairs
      XY = np.array([Xm.ravel(), Ym.ravel()]).T
      # score points and reshape output for plotting
      Z = -gmm.score_samples(XY)
      Z = Z.reshape(Xm.shape)

      # plot contours
      contours = ax.contour(Xm, Ym, Z, norm=LogNorm(vmin=1.0, vmax=100.0),
                             levels=np.logspace(0, 2, 10))

      # plot points
      ax.plot(pos_train_data[:, 0], 
              pos_train_data[:, 1], 
              'b+', label='poisonous', markersize=5)
    
    # add colorbar for the last plot in each figure
    fig.colorbar(contours, shrink=0.8, extend='both')


P4()

### Part 5:

Fit two 4-component full covariance GMMs, one for the positive examples and one for the negative examples in your 2d projected data. Predict the test examples by choosing the label for which the model gives a larger probability (use GMM.score). What is the accuracy?

In [0]:
def P5():
  cov_type='full'
  n_comp=4

  # fit pca and apply dimensionality reduction on X
  pca = PCA(n_components=2)
  reduced_train_data = pca.fit_transform(train_data)

  # convert train labels to bool for use in array indexing
  label_bools = np.array(train_labels, dtype=bool)

  pos_train_data = reduced_train_data[label_bools]
  neg_train_data = reduced_train_data[~label_bools]

  # fit models
  gmm_pos = GaussianMixture(n_components=n_comp, covariance_type=cov_type)
  gmm_pos.fit(pos_train_data)
  gmm_neg = GaussianMixture(n_components=n_comp, covariance_type=cov_type)
  gmm_neg.fit(neg_train_data)

  # get scores for each sample and model
  pos_scores = gmm_pos.score_samples(reduced_train_data)
  neg_scores = gmm_neg.score_samples(reduced_train_data)

  # predict labels
  train_labels_pred = [1 if score[0] >= score[1] else 0 for score 
                       in zip(pos_scores, neg_scores)]

  # calculate accuracy
  accuracy = metrics.accuracy_score(train_labels, train_labels_pred)
  print('The accuracy is ' + str(accuracy))

P5()

### Part 6:

Ideally, we'd like a model that gives the best accuracy with the fewest parameters. Run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters. For example, with 3 PCA components and 2-component diagonal covariance GMMs, you'd have:

( (3 mean vector + 3 covariance matrix) x 2 components ) x 2 classes = 24 parameters

You should vary the number of PCA components, the number of GMM components, and the covariance type.

In [0]:
def get_num_params(n_pca, n_gmm, gmm_type, n_classes):
  # each gaussian's mean is described by an n_pca-dimensional vector
  mean_vector_comps = n_pca * n_gmm

  # depending on covariance matrix type degrees of freedom are different
  if gmm_type == 'full':
    # number dof of n_gmm n_pca-dimensional symmetric matrices
    n_covariance_dof = n_gmm * (n_pca * (n_pca + 1) / 2)
  elif gmm_type == 'diag':
    # each matrix is diagonal so only n_pca degrees of freedom per matrix
    n_covariance_dof = n_gmm * n_pca
  elif gmm_type == 'spherical':
    # each component only has a single variance parameter
    n_covariance_dof = n_gmm
  elif gmm_type == 'tied':
    # components share a covariance matrix
    n_covariance_dof = n_pca * (n_pca + 1) / 2
  else:
    raise ValueError("gmm_type must take a value in " + 
                     "['full', 'diag', 'spherical', 'tied']")

  # multiply by n_classes because separate models for each class
  return int(n_classes * (mean_vector_comps + n_covariance_dof))
  
class PcaGmmModel:
  def __init__(self, n_pca, n_gmm, gmm_type, n_classes, accuracy):
    self.n_pca = n_pca
    self.n_gmm = n_gmm
    self.gmm_type = gmm_type
    self.accuracy = accuracy
    self.num_params = get_num_params(n_pca, n_gmm, gmm_type, n_classes)

  def __str__(self):
    return ('Accuracy = {:.4f}'.format(self.accuracy) + ', num_params = ' 
            + str(self.num_params) + ', n_pca = ' + str(self.n_pca)
            + ', n_gmm = ' + str(self.n_gmm) + ', gmm_type = ' + self.gmm_type)

def P6():
  max_params = 50

  pca_comps = [2, 3, 4]
  gmm_comps = [2, 3, 4]
  gmm_types = ['spherical', 'diag', 'tied', 'full']

  # convert train labels to bool for use in array indexing
  label_bools = np.array(train_labels, dtype=bool)

  # initialize vector to hold output
  models = []

  # loop through hyper-params
  for pca_comp in pca_comps:
    # instantiate PCA and extract reduced positive/negative data sets
    pca = PCA(n_components=pca_comp)
    reduced_train_data = pca.fit_transform(train_data)
    pos_train_data = reduced_train_data[label_bools]
    neg_train_data = reduced_train_data[~label_bools]
    for gmm_comp in gmm_comps:
      for gmm_type in gmm_types:
        # check if we have too many params and skip if we do
        num_params = get_num_params(pca_comp, gmm_comp, gmm_type, 2)
        if num_params > max_params:
          print('pca_comp = ' + str(pca_comp) + ', gmm_comp = ' + str(gmm_comp)
                + ", gmm_type = '" + gmm_type + "' produced " + str(num_params)
                + ' parameters which is greater than max_params value of ' + 
                str(max_params))
          continue

        # create and fit gmm models
        gmm_pos = GaussianMixture(n_components=gmm_comp, 
                                  covariance_type=gmm_type)
        gmm_pos.fit(pos_train_data)
        gmm_neg = GaussianMixture(n_components=gmm_comp, 
                                  covariance_type=gmm_type)
        gmm_neg.fit(neg_train_data)

        # get scores and predict
        pos_scores = gmm_pos.score_samples(reduced_train_data)
        neg_scores = gmm_neg.score_samples(reduced_train_data)
        train_labels_pred = [1 if score[0] >= score[1] else 0 for score 
                             in zip(pos_scores, neg_scores)]

        # calculate accuracy
        accuracy = metrics.accuracy_score(train_labels, train_labels_pred)

        # add to models
        models.append(PcaGmmModel(pca_comp, gmm_comp, gmm_type, 2, accuracy))

  # print output
  models.sort(key=lambda m: m.accuracy, reverse=True)
  print('\nBest model considered:')
  print(models[0])
  print('\nAll models considered:')
  for model in models:
    print(model)



P6()